In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import ast

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting


bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
notebook_url = 'localhost:8888'

In [215]:
# Read in data
TimeData = pd.read_csv(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetNew.csv"
)

# Prep dataset from data
TimeMelted = pd.melt(
    TimeData,
    id_vars=["Training", "ObjectsReinforced", "Reinforced_side", "Date", "Fly"],
    value_name="Values",
    value_vars=[
        "Visits Left Corner",
        "Durations Left Corner",
        "Visits Right Corner",
        "Durations Right Corner",
        "Visits Top Corner",
        "Durations Top Corner",
        "Visits Left Front",
        "Durations Left Front",
        "Visits Right Front",
        "Durations Right Front",
        "Visits Top Front",
        "Durations Top Front",
    ],
    var_name="Variable",
)

TimeMelted[["Values",]] = TimeMelted[
    [
        "Values",
    ]
].applymap(ast.literal_eval)
TimeMelted = TimeMelted.explode("Values")#.drop_duplicates()#.dropna()#.reset_index()
TimeMelted["EventIndex"] = TimeMelted.groupby(["Fly", "Variable"]).cumcount()
TimeMelted = TimeMelted.drop_duplicates().dropna()

In [216]:

conditions = [
    (TimeMelted["Variable"].str.contains("Durations")),
    (TimeMelted["Variable"].str.contains("Visits")),
]

values = ["Durations", "Visits"]
TimeMelted["Kind"] = np.select(conditions, values)

conditions = [
    (
        TimeMelted["Reinforced_side"].str.contains("Right")
        & (TimeMelted["Variable"].str.contains("Right")
    )
    ),
    (
        TimeMelted["Reinforced_side"].str.contains("Left")
        & (TimeMelted["Variable"].str.contains("Left")
    )
    ),
    (
        TimeMelted["Reinforced_side"].str.contains("Right")
        & (TimeMelted["Variable"].str.contains("Left")
    )
    ),
    (
        TimeMelted["Reinforced_side"].str.contains("Left")
        & (TimeMelted["Variable"].str.contains("Right")
     )
    ),
    (TimeMelted["Variable"].str.contains("Top")
     ),

]

values = [
    "Rewarded Side",
    "Rewarded Side",
    "Punished Side",
    "Punished Side",
    "Empty Side",
]
TimeMelted["Condition"] = np.select(conditions, values)

conditions = [
    (
            (TimeMelted["Variable"].str.contains("Corner"))
    ),
    (
            (TimeMelted["Variable"].str.contains("Front")))

]
values = [
    "Corner",
    "Front",
]
TimeMelted["Location"] = np.select(conditions, values)

TimeMelted = TimeMelted#.reset_index()

In [104]:
print(TimeData["Visits Left Corner"][1])
print(TimeData["Durations Left Corner"][1])

[452, 2689, 4476, 4538, 6189, 6324, 11393, 11760, 20790, 24333, 24453, 32411, 42102, 42900, 43139]
[1, 27, 32, 3, 37, 8, 19, 49, 32, 20, 46, 25, 54, 76, 88]


In [ ]:
print(TimeData["Visits Top Corner Corner"][1])
print(TimeData["Durations Left Corner"][1])

In [86]:
TimeMelted = TimeMelted.stack('Kind')

KeyError: 'Requested level (Kind) does not match index name (None)'

In [112]:
TimeMelted = (
    TimeMelted.reset_index()
    .pivot_table(
        index=["Condition","Fly","EventIndex","Training", "ObjectsReinforced", ],
        columns="Kind",
        values="Values",
    )
    .reset_index()
)

In [113]:

# Produce the subsets
ReinforcedVisits_Corner = TimeMelted[TimeMelted["Condition"] == "Rewarded Corner"]
ReinforcedVisits_Front = TimeMelted[TimeMelted["Condition"] == "Rewarded Front"]
PunishedVisits_Corner = TimeMelted[TimeMelted["Condition"] == "Punished Corner"]
PunishedVisits_Front = TimeMelted[TimeMelted["Condition"] == "Punished Front"]
EmptyVisits_Corner = TimeMelted[TimeMelted["Condition"] == "Empty Corner"]
EmptyVisits_Front = TimeMelted[TimeMelted["Condition"] == "Empty Front"]

In [4]:
# Options for x and y selectors

xy_options = list(
    ReinforcedVisits_Corner.columns[ReinforcedVisits_Corner.columns.isin(["Visits", "Durations"])]
)  #'ObjectsReinforced','Training', 'Fly'

# Define the selector widgets

x_selector = bokeh.models.Select(
    title="x",
    options=xy_options,
    value="Visits",
    width=200,
)

y_selector = bokeh.models.Select(
    title="y",
    options=xy_options,
    value="Durations",
    width=200,
)

colorby_selector = bokeh.models.Select(
    title="color by",
    options=[
        "none",
        "ObjectsReinforced",
        "Training",
        "Fly",
    ],
    value="none",
    width=200,
)

In [ ]:
# Column data source
source = bokeh.models.ColumnDataSource(
    dict(x=ReinforcedVisits_Corner[x_selector.value], y=ReinforcedVisits_Corner[y_selector.value],
         
        )
)

# Add a column for colors; for now, all bokeh's default blue
source.data["color"] = ["#1f77b3"] * len(ReinforcedVisits_Corner)

# Make the plot
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color", 
                  #legend_group="color",
                 )

In [ ]:
from bokeh.palettes import Category10
import itertools

colors = itertools.cycle(Category10[10])

colors

def color_gen():
    yield from itertools.cycle(Category10[10])
color = color_gen()

In [ ]:
colors = []

for fly, c in zip(ReinforcedVisits_Corner["Fly"].unique(), color):
    colors.append(c)

colors
#colors = colorgen() for flies in ReinforcedVisit["Fly"]

In [ ]:
def gfmt_callback(attr, new, old):
    """Callback for updating plot of GMFT results."""
    # Update color column
    if colorby_selector.value == "none":
        source.data["color"] = ["#1f77b3"] * len(ReinforcedVisits_Corner)
        
    elif colorby_selector.value == "ObjectReinforced":
        source.data["color"] = [
            "#1f77b3" if objects == "Blue" else "#ff7e0e"
            for objects in ReinforcedVisits_Corner["ObjectsReinforced"]
        ]
        
    elif colorby_selector.value == "Training":
        source.data["color"] = [
            "#1f77b3" if Training == "Trained" else "#ff7e0e"
            for Training in ReinforcedVisits_Corner["Training"]
        ]

    elif colorby_selector.value == "Fly":
        source.data["color"] = [
            
            colors for flies in ReinforcedVisits_Corner["Fly"].unique()
        ]

    # Update x-data and axis label
    source.data["x"] = ReinforcedVisits_Corner[x_selector.value]
    p.xaxis.axis_label = x_selector.value

    # Update x-data and axis label
    source.data["y"] = ReinforcedVisits_Corner[y_selector.value]
    p.yaxis.axis_label = y_selector.value


# Connect selectors to callback
colorby_selector.on_change("value", gfmt_callback)
x_selector.on_change("value", gfmt_callback)
y_selector.on_change("value", gfmt_callback)

# Build the layout
gfmt_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

In [ ]:
def gfmt_app(doc):
    doc.add_root(gfmt_layout)

bokeh.io.show(gfmt_app, notebook_url=notebook_url)

In [ ]:
# Column data source
source = bokeh.models.ColumnDataSource(
    dict(x=ReinforcedVisits_Corner["Visits"], y=ReinforcedVisits_Corner["Durations"],
         
        )
)

# Add a column for colors; for now, all bokeh's default blue
source.data["color"] = ["#1f77b3"] * len(ReinforcedVisits_Corner)

# Make the plot
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color", 
                  #legend_group="color",
                 )

bokeh.io.show(p)

In [159]:
def gfmt_callback(attr, new, old):
    """Callback for updating plot of GMFT results."""
    # Update color column
    if colorby_selector.value == "none":
        source.data["color"] = ["#1f77b3"] * len(ReinforcedVisits_Corner)
        
    elif colorby_selector.value == "ObjectReinforced":
        source.data["color"] = [
            "#1f77b3" if objects == "Blue" else "#ff7e0e"
            for objects in ReinforcedVisits_Corner["ObjectsReinforced"]
        ]
        
    elif colorby_selector.value == "Training":
        source.data["color"] = [
            "#1f77b3" if Training == "Trained" else "#ff7e0e"
            for Training in ReinforcedVisits_Corner["Training"]
        ]

    elif colorby_selector.value == "Fly":
        source.data["color"] = [
            
            colors for flies in ReinforcedVisits_Corner["Fly"].unique()
        ]

    # Update x-data and axis label
    source.data["x"] = ReinforcedVisits_Corner[x_selector.value]
    p.xaxis.axis_label = x_selector.value

    # Update x-data and axis label
    source.data["y"] = ReinforcedVisits_Corner[y_selector.value]
    p.yaxis.axis_label = y_selector.value


# Connect selectors to callback
colorby_selector.on_change("value", gfmt_callback)
x_selector.on_change("value", gfmt_callback)
y_selector.on_change("value", gfmt_callback)

# Build the layout
gfmt_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

In [161]:
def gfmt_app(doc):
    doc.add_root(gfmt_layout)

bokeh.io.show(gfmt_app, notebook_url=notebook_url)

In [114]:
# Column data source
source = bokeh.models.ColumnDataSource(
    dict(x=ReinforcedVisits_Corner["Visits"],
         y=ReinforcedVisits_Corner["Durations"],
         Fly=ReinforcedVisits_Corner['Fly'],
         Training=ReinforcedVisits_Corner['Training'],
         ObjectsReinforced=ReinforcedVisits_Corner['ObjectsReinforced']
         
        )
)

# Add a column for colors; for now, all bokeh's default blue
source.data["color"] = ["#1f77b3"] * len(ReinforcedVisits_Corner)

source.data["color"]

['#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1

In [115]:
# Make the plot
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color",
                  #legend_group="color",
                 )

bokeh.io.show(p)

In [118]:
from bokeh.palettes import Category10
import itertools

colors = itertools.cycle(Category10[10])

colors

def color_gen():
    yield from itertools.cycle(Category10[10])
color = color_gen()

In [123]:
Flycolors = []

for fly, c in zip(ReinforcedVisits_Corner["Fly"].unique(), color):
    Flycolors.append(c)

Flycolors
#colors = colorgen() for flies in ReinforcedVisit["Fly"]

['#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22']

In [194]:
Fly = ReinforcedVisits_Corner['Fly'].astype('string').unique().tolist()


KeyError: 2150

In [212]:

source.data['color'] = bokeh.transform.factor_cmap(source.data['color'], palette=bokeh.palettes.Viridis256, factors=Fly)

ValueError: failed to validate ColumnDataSource(id='2150', ...).data: expected an element of ColumnData(String, Seq(Any)), got {'color': {'field': {'field': 8170     2
8171     2
8172     2
8173     2
8174     2
        ..
9769    61
9770    61
9771    61
9772    61
9773    61
Name: Fly, Length: 1604, dtype: int64, 'transform': CategoricalColorMapper(id='3243', ...)}, 'transform': CategoricalColorMapper(id='3277', ...)}}

In [210]:
source.data["color"] = index_cmap

ValueError: failed to validate ColumnDataSource(id='2150', ...).data: expected an element of ColumnData(String, Seq(Any)), got {'color': {'field': 8170     2
8171     2
8172     2
8173     2
8174     2
        ..
9769    61
9770    61
9771    61
9772    61
9773    61
Name: Fly, Length: 1604, dtype: int64, 'transform': CategoricalColorMapper(id='3243', ...)}}

In [211]:
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color=index_cmap,
                  #legend_group="color",
                 )

bokeh.io.show(p)

ValueError: failed to validate Circle(id='3276', ...).hatch_color: expected an element of either String, Dict(Enum('expr', 'field', 'value', 'transform'), Either(String, Instance(Transform), Instance(Expression), Nullable(Color))) or Nullable(Color), got {'field': 8170     2
8171     2
8172     2
8173     2
8174     2
        ..
9769    61
9770    61
9771    61
9772    61
9773    61
Name: Fly, Length: 1604, dtype: int64, 'transform': CategoricalColorMapper(id='3243', ...)}

In [127]:
source.data["color"] = [
            "#1f77b3" if objects == "Blue" else "#ff7e0e"
            for objects in ReinforcedVisits_Corner["ObjectsReinforced"]
        ]

source.data["color"]

['#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#f

In [128]:
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
    tooltips=[('fly', '@{Fly}'),
              ('Training state', '@Training'),
              ('Object reinforced', '@ObjectsReinforced')
             ],
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color",
                  #legend_group="color",
                 )

bokeh.io.show(p)

In [225]:
Categories = TimeMelted["Condition"].unique().tolist() + ['All']